# SIFT & Cluster Ensemble Model Validation

This script runs validations for the mentioned ensemble models in the report, namely SIFT, CLOCK, Combination Model and SHOCK

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import cv2
from tqdm import tqdm
from kmeans import onl_kmeans, hist_onl_kmeans

In [4]:
dist = pd.read_csv('./resnet101_matches_distance.csv').values
fname = pd.read_csv('./resnet101_matches_filename.csv').values

In [20]:
good = pd.read_csv('./sift_matches_distance.csv').values
fgood = pd.read_csv('./sift_matches_filename.csv').values

In [23]:
train_xy = pd.read_csv('validate_train.csv', index_col=0)
train_path =  pd.read_csv('validate_train.csv')['id'].values
test_path = pd.read_csv('validate.csv')['id'].values
test_xy =  pd.read_csv('validate.csv')

In [7]:
# Limit the candidate pictures to a limited decrease only, comparing to the top one
extract_match = lambda i, thresh: fname[i,np.argwhere(dist[i] < dist[i,0] + thresh)]

## SIFT Implementation: Only Match on sparse clusters

In [10]:
# Processing
threshold = 5
max_clusters = 5
max_radius = 7
min_size = 3

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    coords = train_xy.loc[img_idx].values
    _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
    if centroid is None:
        with open(f'./train_kp/train_kp{test}.pckl', 'rb') as test_sift_file:
            des_test = pickle.load(test_sift_file)
        goods = []

        for train in img_idx:
            with open(f'./train_kp/train_kp{train}.pckl', 'rb') as train_sift_file:
                des_train = pickle.load(train_sift_file)

            # Matching descriptor using KNN algorithm
            if des_train is None or len(des_train) < 2:
                goods.append(-1)
                continue
            matches = flann.knnMatch(des_test,des_train,k=2)

            # Store all good matches as per Lowe's Ratio test.
            good = len([m for m,n in matches if m.distance < 0.7*n.distance])
            goods.append(good)
        
        max_idx = np.argmax(goods)
        centroids.append(train_xy.loc[img_idx[max_idx]].values)
    else:
        centroids.append(centroid)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [03:19<00:00,  3.01it/s]


In [13]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_1.csv',index_label='id',header=['x','y'])

In [16]:
CLOCK_predict = pd.read_csv('CLOCK_1.csv')

# calculate MAE
MAE = np.abs(CLOCK_predict['x']-test_xy['x']) +  np.abs(CLOCK_predict['y']-test_xy['y'])
MAE = np.sum(MAE)/1200
MAE

7.969589224016131

## (Combination model) Exhaustive SIFT implementation: Do feature matching on all candidated pools

In [17]:
# COCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 7
min_size = 1

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    
    with open(f'./train_kp/train_kp{test}.pckl', 'rb') as test_sift_file:
        des_test = pickle.load(test_sift_file)
    goods = []

    # Weak finding: Do COCK instead
    if des_test is None or len(des_test) < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
        centroids.append(centroid)
        continue

    for train in img_idx:
        with open(f'./train_kp/train_kp{train}.pckl', 'rb') as train_sift_file:
            des_train = pickle.load(train_sift_file)

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
    
    max_idx = np.argmax(goods)
    centroids.append(train_xy.loc[img_idx[max_idx]].values)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [09:57<00:00,  1.00it/s]


In [18]:
out = pd.DataFrame(centroids,index=test_path)
out.to_csv('CLOCK_2.csv',index_label='id',header=['x','y'])


In [19]:
CLOCK_predict = pd.read_csv('CLOCK_2.csv')

# calculate MAE
MAE = np.abs(CLOCK_predict['x']-test_xy['x']) +  np.abs(CLOCK_predict['y']-test_xy['y'])
MAE = np.sum(MAE)/1200
MAE

7.0321777778911105

## (Combination model + CLOCK) Exhaustive SIFT implmentation: Do feature matching on CNN candidate pool, then do clustering to odd out outliers

In [ ]:
# COCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 3
max_radius = 7
min_size = 1
max_match_keep = 0.3

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    
    
    f = open(f'./train_kp/train_kp{test}.pckl', 'rb')
    des_test = pickle.load(f)
    f.close()
    
    goods = []
    current_good = 0
    
    for train in train_path:
        f = open(f'./train_kp/train_kp{train}.pckl', 'rb')
        des_train = pickle.load(f)
        f.close()

        # If matching is weak, it must be irrelevant. Assign -1 to no. matches
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)
        # Store all good matches based on Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)

    # Sorting. The order starts from the least matches
    sorted_idx = np.argsort(goods)
    sorted_path = [train_path[idx] for idx in sorted_idx]
    sorted_goods = [goods[idx] for idx in sorted_idx]
    
    sift_fname = sorted_path
    sift_match = sorted_goods

    # Weak finding: Do COCK instead
    if sift_fname[0] is np.nan:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = onl_kmeans(coords, img_idx, 5, max_radius, min_size)
        centroids.append(centroid)
        continue

    # Basically get all cnn indices in order of best SIFT matches
    matchings = []
    good_m = []
    for match_idx,m in enumerate(sift_fname):
        if m in img_idx:
            matchings.append(m)
            good_m.append(sift_match[match_idx])
    # Once again do thresholding
    good_match = [m for idx,m in enumerate(matchings) 
                  if good_m[idx] > good_m[0]*max_match_keep]

    # Weak finding: Do COCK instead
    if good_m[0] < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = onl_kmeans(coords, img_idx, 5, max_radius, min_size)
        centroids.append(centroid)
        continue

    coords = train_xy.loc[good_match].values
    _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
    
    centroids.append(centroid)

In [28]:
out = pd.DataFrame(centroids,index=test_path[:131])
out.to_csv('CLOCK_3.csv',index_label='id',header=['x','y'])

In [40]:
CLOCK_predict = pd.read_csv('CLOCK_3.csv')

# calculate MAE
MAE = np.abs(CLOCK_predict['x']-test_xy['x']) +  np.abs(CLOCK_predict['y']-test_xy['y'])
MAE = np.sum(MAE)/ (400)
MAE

7.558215594869987

## Exhaustive SIFT implementation: Do feature matching on all candidated pools, choose best match

In [ ]:
# COCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 7
min_size = 1

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    img_idx = extract_match(i,threshold).flatten()
    
    with open(f'./train_kp/train_kp{test}.pckl', 'rb') as test_sift_file:
        des_test = pickle.load(test_sift_file)
    goods = []

    # Weak finding: Do COCK instead
    if des_test is None or len(des_test) < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = onl_kmeans(coords, img_idx, max_clusters, max_radius, min_size)
        centroids.append(centroid)
        continue

    for train in img_idx:
        with open(f'./train_kp/train_kp{train}.pckl', 'rb') as train_sift_file:
            des_train = pickle.load(train_sift_file)

        # Matching descriptor using KNN algorithm
        if des_train is None or len(des_train) < 2:
            goods.append(-1)
            continue
        matches = flann.knnMatch(des_test,des_train,k=2)

        # Store all good matches as per Lowe's Ratio test.
        good = len([m for m,n in matches if m.distance < 0.7*n.distance])
        goods.append(good)
    
    max_idx = np.argmax(goods)
    centroids.append(train_xy.loc[img_idx[max_idx]].values)

out = pd.DataFrame(centroids,index=test_path)
N=1800
out.to_csv('Combi_1.csv',index_label='id',header=['x','y'])

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [10:07<00:00,  1.01s/it]


In [ ]:
SHOCK_predict = pd.read_csv('Combi_1.csv')

# calculate MAE
MAE = np.abs(SHOCK_predict['x']-train_xy['x']) + np.abs(SHOCK_predict['y']-train_xy['y'])
MAE = np.sum(MAE)/N
MAE

6.701867815924616

## (Combination + SHOCK) Exhaustive SIFT, vote between best match, biggest cluster and most similar color spectrum

In [ ]:
# COCK params for images with few features
MIN_MATCHES = 5
threshold = 5
max_clusters = 5
max_radius = 15
min_size = 1
max_match_keep = 0.4

# FLANN specs
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params,search_params)

centroids = []
for i,test in enumerate(tqdm(test_path)):
    test_img = cv2.imread('./test/' + test + '.jpg')
    t_hist = cv2.calcHist([test_img],[0],None,[256],[0,256])

    img_idx = extract_match(i,threshold).flatten()
    
    sift_fname = fgood[i,:]
    sift_match = good[i,:]

    # Weak finding: Do COCK instead
    if sift_fname[0] is np.nan:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, max_radius, min_size, take_best_hist=True)
        centroids.append(centroid)
        continue

    # Basically get all cnn indices in order of best SIFT matches
    matchings = []
    good_m = []
    for match_idx,m in enumerate(sift_fname):
        if m in img_idx:
            matchings.append(m)
            good_m.append(sift_match[match_idx])
    # Once again do thresholding
    good_match = [m for idx,m in enumerate(matchings) 
                  if good_m[idx] > good_m[0]*max_match_keep]

    # Weak finding: Do COCK instead
    if good_m[0] < MIN_MATCHES:
        coords = train_xy.loc[img_idx].values
        _, _, centroid = hist_onl_kmeans(coords, t_hist, img_idx, -1, max_radius, min_size, take_best_hist=True)
        centroids.append(centroid)
        continue

    coords = train_xy.loc[good_match].values
    _, _, centroid = hist_onl_kmeans(coords, t_hist, good_match, max_clusters, max_radius, min_size)
    
    centroids.append(centroid)

out = pd.DataFrame(centroids,index=test_path)
N=1800
out.to_csv('SHOCK_2.csv',index_label='id',header=['x','y'])

100%|██████████| 1200/1200 [01:22<00:00, 14.52it/s]


In [ ]:
SHOCK_predict = pd.read_csv('SHOCK_2.csv')

# calculate MAE
MAE = np.abs(SHOCK_predict['x']-train_xy['x']) + np.abs(SHOCK_predict['y']-train_xy['y'])
MAE = np.sum(MAE)/N
MAE

4.592507407498518